## Problem 2: Calculate and visualise the *dominance areas* of shopping centres (10 points)

In this problem, the aim is to delineate the dominance area of each shopping centre. For this 
exercise, we define the ‘dominance area’ of a shopping centre as the area from which it can 
be reached faster than other shopping centres. We will use public transport travel times.

![](images/dominance_areas_example_1000x500px.png)

*Sample result: a map showing the areas of dominance of each shopping centre, and the travel 
times to the closest shopping centre in the entire metropolitan area*



---

### Data

The **input data** is identical to what you have used for *problem 1*, 
see [there](./Exercise-4-problem-1.ipynb) for detailed data descriptions.

---


### An overview of the tasks

This task comprises of three major subtasks. In contrast to earlier exercises, we 
do not provide a detailed, step-by-step ‘cooking recipe’. Rather, you are free to
implement the necessary steps in any order you see fit, and choose any variable
names of your liking. 

To test intermediate results, implement `assert` statements, output the `head()`
of a data frame, or plot the data. Remember to add comments to all of your code,
so future you (and us) can understand what each section does.

The **only strict requirement** is the **file name** of
the **output** map plot: `DATA_DIRECTORY / "dominance_areas.png"`.

1. Load the YKR grid and the individual travel time data sets, and combine them 
   into one geo data frame. This is essentially the same as *problem 1*, except
   that you must load all eight shopping centre data files.
2. Find the closest shopping centre to each grid cell. In the combined data set,
   find the minimum travel time to any of the shopping centres, save the value in
   a new column, and shopping centre name in another new column.
   See the [hints](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-4/exercise-4.html#hints)
   to this exercise for a suggestions on how to achieve this 
   ([`pandas.DataFrame.min()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.min.html) and
   [`pandas.DataFrame.idxmin()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmin.html)
   will be helpful)
3. Visualise the dominance areas and travel times. Use 2⨉1 subplots to plot
   the most dominant (closest) shopping centre for each grid cell, and the
   travel time to the closest shopping centre for each grid cell.

---


In [1]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [2]:
# ADD YOUR OWN CODE HERE
# Imports
import geopandas as gpd
import pandas as pd
import numpy as np
import mapclassify
import matplotlib.pyplot as plt
import glob

/tmp/ipykernel_32729/1794029931.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [17]:
# Read grid
grid = gpd.read_file(DATA_DIRECTORY/"YKR_grid_EPSG3067.gpkg")

# Read csv files
# Get filenames
txt_files = glob.glob(DATA_DIRECTORY.as_posix() + "/" + "*.txt")

# Create a dictionary of df's for each filename
dfs = {}

for file in txt_files:
    # Read the .txt file into a DataFrame
    df = pd.read_csv(file, sep=";")
    df = df[['from_id', 'pt_r_t']]
    df["pt_r_t"] = df["pt_r_t"].replace(-1, np.nan)
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    file_name = file.split("/")[-1].split("_")[-1][:-4]
    df.rename(inplace=True, columns={"pt_r_t":f"pt_r_t_{file_name}"})
    grid = grid.set_index("YKR_ID").join(df.set_index("from_id"), rsuffix=f"_{file_name}").reset_index()

grid.dropna(inplace=True)
grid.reset_index(inplace=True, drop=True)

In [18]:
# ADD YOUR OWN CODE HERE
grid

,YKR_ID,geometry,pt_r_t_Jumbo,pt_r_t_Omena,pt_r_t_Ruoholahti,pt_r_t_Myyrmanni,pt_r_t_Dixi,pt_r_t_Itis,pt_r_t_Forum
0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6...",101.0,141.0,118.0,90.0,102.0,132.0,110.0
1,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6...",108.0,143.0,121.0,93.0,109.0,135.0,113.0
2,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6...",109.0,145.0,123.0,95.0,111.0,137.0,115.0
3,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6...",114.0,149.0,127.0,99.0,115.0,141.0,119.0
4,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6...",98.0,134.0,111.0,83.0,99.0,125.0,103.0
...,...,...,...,...,...,...,...,...,...
13015,6016697,"POLYGON ((372750.000 6665500.000, 372500.000 6...",111.0,40.0,50.0,97.0,81.0,81.0,54.0
13016,6016698,"POLYGON ((373000.000 6665500.000, 372750.000 6...",109.0,38.0,48.0,95.0,79.0,79.0,52.0
13017,6016699,"POLYGON ((373250.000 6665500.000, 373000.000 6...",110.0,40.0,50.0,97.0,81.0,81.0,54.0
13018,6018253,"POLYGON ((372750.000 6665250.000, 372500.000 6...",114.0,43.0,53.0,100.0,84.0,84.0,57.0


In [ ]:
# ADD YOUR OWN CODE HERE


In [ ]:
# NON-EDITABLE TEST CELL
# Check that output figure file exists
assert (DATA_DIRECTORY / "dominance_areas.png").exists()


--- 

**Do not forget to plot the result map, and save it to `DATA_DIRECTORY / "dominance_areas.png"`!**

---

## Reflections

This was a significantly more complex exercise that previous ones, and it included finding
a solution yourself. 

- What was most difficult part? 
- Where did you get stuck? 
- What was the easiest, and
- what was the most fun part of this exercise?

Add your answer below


---

## Well done!

Congratulations, you completed exercise 4. Good Job!